In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Read data

In [2]:
filename = "pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_csv(filename, names=names)

data.head(20)

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


In [3]:
data.shape

(768, 9)

In [4]:
data.dtypes

preg       int64
plas       int64
pres       int64
skin       int64
test       int64
mass     float64
pedi     float64
age        int64
class      int64
dtype: object

### Separate dataset

In [5]:
data = data.values

In [6]:
X = data[:, 0:8]
Y = data[:, 8]

### Split dataset

In [7]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=10, random_state=7)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=7)

### Model

In [9]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [10]:
model.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Accuracy
- only suitable when there are an equal number of observations in each class (which is rarely the case)
- all predictions and prediction errors are equally important, which is often not the case

In [11]:
from sklearn.model_selection import cross_val_score
results = cross_val_score(model, X, Y, cv=kfold, scoring='accuracy')

print("Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))

Accuracy: 0.770 (0.048)


### Logarithmic Loss
-  evaluating the predictions of probabilities of membership to a given class
- The scalar probability between 0 and 1 can be seen as a measure of confidence for a prediction by an algorithm
- Predictions that are correct or incorrect are rewarded or punished proportionally to the confidence of the prediction
- Smaller logloss is better with 0 representing a perfect logloss

In [12]:
from sklearn.model_selection import cross_val_score
results = cross_val_score(model, X, Y, cv=kfold, scoring='neg_log_loss')

print("Logloss: %.3f (%.3f)" % (results.mean(), results.std()))

Logloss: -0.493 (0.047)


### Area Under ROC Curve
- a performance metric for binary classification problems
-  represents a model’s ability to discriminate between positive and negative classes
    - An area of 1.0 represents a model that made all predictions perfectly
    - An area of 0.5 represents a model that is as good as random
- ROC can be broken down into sensitivity and specificity
- A binary classification problem is really a trade-off between sensitivity and specificity.(Recall - Precision)
    -  Sensitivity is the true positive rate also called the recall. It is the number of instances from the positive (first) class that actually predicted correctly.
    - Specificity is also called the true negative rate. Is the number of instances from the negative (second) class that were actually predicted correctly.

In [13]:
from sklearn.model_selection import cross_val_score

results = cross_val_score(model, X, Y, cv=kfold, scoring='roc_auc')
print("AUC: %.3f (%.3f)" % (results.mean(), results.std()))

AUC: 0.824 (0.041)


### Confusion Matrix

In [14]:
from sklearn.metrics import confusion_matrix

predicted = model.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
print(matrix)

[[141  21]
 [ 41  51]]


### Classification Report

In [15]:
from sklearn.metrics import classification_report

predicted = model.predict(X_test)
report = classification_report(Y_test, predicted)
print(report)

             precision    recall  f1-score   support

        0.0       0.77      0.87      0.82       162
        1.0       0.71      0.55      0.62        92

avg / total       0.75      0.76      0.75       254

